**Name:** \_\_\_\_\_Huanchen Wang\_\_\_\_\_

**EID:** \_\_\_\_\_57558749\_\_\_\_\_

**Kaggle Team Name:** \_\_\_\_\_unknownwang\_\_\_\_\_

# CS5489 - Assignment 1 - Movie Rating Prediction

## Final submission
In this file, put the code that generates your final Kaggle submission. It will be used to verify that your Kaggle submission is reproducible.

In [1]:
%matplotlib inline
import matplotlib_inline   # setup output image format
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import csv
random.seed(100)

In [2]:
def read_text_data(fname):
    txtdata = []
    revid   = []
    classes = []
    with open(fname, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            # get the text
            txtdata.append(row[0])
            revid.append(row[1])
            # get the class (convert to integer)
            if len(row)>2:
                classes.append(int(row[2]))
    
    if (len(classes)>0) and (len(txtdata) != len(revid)):        
        warn.error("mismatched length!")
    
    return (txtdata, revid, classes)

def write_csv_kaggle_sub(fname, Y):
    # fname = file name
    # Y is a list/array with class entries
    
    # header
    tmp = [['Id', 'Prediction']]
    
    # add ID numbers for each Y
    for (i,y) in enumerate(Y):
        tmp2 = [(i+1), y]
        tmp.append(tmp2)
        
    # write CSV file
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(tmp)

In [3]:
# load the data
(traintxt, trainrevid, trainY) = read_text_data("movierating_train.txt")
(testtxt, testrevid, _)        = read_text_data("movierating_test.txt")

print(len(traintxt))
print(len(testtxt))

4006
1000


In [ ]:
### INSERT YOUR CODE HERE

In [4]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
# nltk.download()
import datasets

In [5]:
def tokenize_word(doc):
    # get stopwords list
    stop_words = stopwords.words('english')
    # tokenize the text each word initially
    tokens = nltk.word_tokenize(doc)
    # remove the punctuation and stopwords in the text
    tokens = [word for word  in tokens if word not in stop_words and word not in punctuation]
    return ' '.join(tokens)

traintxt = [tokenize_word(doc) for doc in traintxt]
testtxt = [tokenize_word(doc) for doc in testtxt]

dataset= datasets.Dataset.from_dict({'text':traintxt, 'label':trainY})


In [7]:
# from datasets import load_metric
import torch
use_gpu = torch.cuda.is_available()
import evaluate
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True,max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset, eval_dataset = tokenized_datasets.train_test_split(test_size=0.2)

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                            num_labels=4)
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()